# Class 5 - 20.4.20

# Assignment 3 Comments

## Short and Precise Functions

As we said earlier this year, our goal in this course is not only to learn Python, it's also to learn how to write better code. As you all know, one of best methods to solve problems is "divide and conquer". By this I mean that as a first step when encountering a new problem - a HW assignment or a scientific question - is to divide it into smaller parts and solve each part separately. This method works in our day to day research, and it's also extremely important when writing code.

In code, these "smaller parts" are obviously functions or class methods. In MATLAB this wasn't encouraged too much, since until very recently you were unable to create a file containing more than one function, or a function and a script. This meant that if you wanted to "divide and conquer" your algorithm, by the time you're done you'd end up with hundreds of files, some of them only containing a few lines of code. In Python this technical difficulty doesn't exist - you may write as many functions as you'd like in a single file, and `import` some (or all) of them to other files and modules.

Since the technical difficulties aren't really an issue, let's drill down by looking at a slightly more concrete example. Assume I need to write the pre-processing steps for some cell counting software\algorithm that someone else wrote for me. The first thing I do when trying to solve this type of puzzle is to decompose it into its main parts. Then I'll go over each part and de-structure it further, until I'm left with small enough units of work, which will become individual functions. In principle, the thought process may follow the following mock diagram:

![Examplary modular approach (made with Diagram.Codes)](extra_material/cell_counting_algo_line_of_thought.svg)

Here each rectangle is a unit of work I need to do, and I sometimes add the function name that connects a rectangle to the next one, if I feel like this information is useful. The different steps show how I progressed from having a general concept of the software I'm about to write, to having a pretty concrete graph which I should "copy-paste" into my code, in the form of functions, of course. I.e., all that's left for me to do after writing the graph explicitly like that is to actually implement the code. This process, at the beginning, won't be too natural, but after a few times I'm sure it'll be much easier for you, perhaps to a point in which for most algorithms you don't even need to draw them out on paper.

The important thing to remember here is that each component or function should do exactly one thing, and do it well. Even if that thing is very small, and\or very simple, it should still be its own function.

Moving on to real examples, I want you to see the final product of this thought process as it manifested in the solution to HW2, question 1. As a reminder, the question was the "Morse Code" question, in which you were given a dictionary mapping letters to code, and were asked to translate a text to it. My final diagram for the main function (`english_to_morse`) was the following:

![English to morse diagram](extra_material/english_to_morse_diagram.svg)

And this is the code that I published as a solution (docstring removed):

```python
def english_to_morse(
    input_file: str = "lorem.txt", output_file: str = "lorem_morse.txt"
):
    file = pathlib.Path(str(input_file))
    if not file.exists():
        raise ValueError(f"File {file} doesn't exist.")
    # Generate a new dictionary with all letters, upper and lower case
    code_ord_upper = {ord(key): val for key, val in MORSE_CODE.items()}
    code_ord_lower = {ord(key.lower()): val for key, val in MORSE_CODE.items()}
    new_code = {**code_ord_lower, **code_ord_upper}
    data = convert(file, new_code)
    to_disk(data, pathlib.Path(output_file))
```

As you can see, not all rectangles are independent functions - although they definitely could be. For one reason or the other I decided to keep some of the code inside the main function, and move some other parts to an independent function. Usually the underlying reason has to do with code reusability, or the question "how likely it is that I'll re-use this code snippet somewhere else?". If the answer is "likely" then I'd take that chunk of code and extract it to a function, since later I might have to write another piece of code that needs the exact same functionality. In our current case, I decided that writing out to disk is a "general" function that could be re-used elsewhere.

If reusability is the guiding principle, why did I factor out `convert`? Well, because there are other considerations. In the `convert` case, I decided that the reader of `english_to_morse` needs to see the full picture of the function, and some details might be of lesser interest. The `convert` function uses very clear arguments (the data and the dictionary) and outputs a new data variable, so a naive reader can easily understand what's going on here without checking exactly how am I converting the file. Besides, the actual `convert` operation might be costly and complicated, and these details will "bury" the logic and flow of `english_to_morse`, decreasing readability even further.

As a final comment, I wish to remind you that both the `convert` and the `to_disk` functions are extremely short. Here's `to_disk`, docstring removed:

```python
def to_disk(data: str, file: pathlib.Path):
    try:
        with open(file, "w") as f:
            f.write(data)
    except PermissionError as e:
        print(f"Error: {e}. Data is still accessible in 'data' variable.")
```

All this function does is write something to disk, and report if it fails. These five lines of code could easily fit inside `english_to_morse`, but moving this unit of work into its own function improves readability and reusability of our code, which are two huge pluses.

To summarize, I found the smallest components of `english_to_morse`, written an outline of how would my solution look like, and finally I coded it together. Later on in the course we'll see two interesting aspects in which this process could be better.

## General Comments (Assignment 2 cont'd.)

1. Remember to use `with` blocks to open files, just as I did above.
2. Don't define functions within functions, i.e. avoid this:
    ```python
def a(b):
    c = b + 1
    def add_one(var):
        return var + 1
    c = add_one(c)
    return c
    ```
  It's very rare that you'll need a function inside another function - it's almost always better to define it in the outer scope, and use it inside the other function (`a` in this case).
3. `import` statement belong to the top of the file.
4. Don't comment each line. If you chose proper function and variable names, most of your code should be self exlplanatory. Write long docstrings, and if there's a chunk of code that needs explain, give a "what is going on here, and why" overview before that chunk of code, not a detailed line-by-line account. You can check the `english_to_morse` function above for an example.
5. `return`ing from a function can be done only once! This will always return 1:
```python
def a():
    return 1
    return 2
```
    If you want to return both numbers, write `return (1, 2)`. Notice that i added parenthesis to make it a tuple, but if you don't want to return a tuple you don't need to wrap things in parenthesis.

# Scientific Python

## Introduction

![The scientific Python stack](extra_material/scientific_python.jpg)

The onion-like scientific stack of Python is composed of the most important packages used in the Python scientific world. Here's a quick overview of the most important names in the stack, before we dive in deep:
1. IPython: A REPL, much like the command window in MATLAB. Lets you write Python on the fly, debug and check the performance of your code, and much (much!) more.
2. NumPy: Standard implementation of multi-dimensional arrays in Python. 
3. Jupyter: Notebooks aimed at data exploration. Write code and see its output immediately, with useful rich text (Markdown) annotations to accompany it - just like this notebook.
4. SciPy: Implements advanced algorithms widely used in scientific programming. Advanced linear algebra, data structures, curve fitting, signal processing, modeling and more.
5. Matplotlib: Visualizations of your data in Python with MATLAB-like interface.
6. Pandas: Tabular data manipulation.
7. Seaborn: Smart visualizations of tabular data.

This scientific stack is one of the main reasons that empowered Python in recent years to the prominent spot it has today. Throughout the course you will become familiar with all ofthe mentioned tools, and more.

## NumPy

NumPy is the de-facto implementation of arrays in Python. They're very similar to MATLAB arrays (and to other implementations in other programming languages), which will hopefully make us feel comfortable with this module.

>[This](https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html) page contains a thorough comparison of similar functions in MATLAB and numpy. If you want to use MATLAB's `find` function and can't seem to find a counterpart in numpy (it's `nonzero`, BTW), or if you wish to see how do different arithmetic functions compare between the two, that page can be of great assistance.

Although `numpy` is used extensively in the Python eco-system, it doesn't come with the Python standard library, which means you have to install it first. Activate your environment and run `pip install numpy` from the command line (inside VSCode, for example). Only then you can follow the rest of the class.

In [1]:
import numpy as np  # standard alias for numpy

In [2]:
l = [1, 2, 3]
arr = np.array(l)  # obviously equal to np.array([1, 2, 3])
arr

array([1, 2, 3])

`arr` is a numpy array. Notice how `numpy` has to take existing data, in this case in the form of a list `l`, and convert it into an array. In MATLAB everything is an array all the time so we don't think of this conversion, but here the conversion happens explicitly when you call `np.array()` on some collection of data. 

Let's continue to examine the basic properties of this array.

In [3]:
# Check the dimensionality of the array:
print("Shape: ", arr.shape)
print("NDim: ", arr.ndim)
# But...
a = np.array(1)  # wrong instatiation! Don't use it, instead write np.array([1])
print("Wrong instantiation results:")
print("Shape: ", a.shape)
print("NDim: ", a.ndim)

Shape:  (3,)
NDim:  1
Wrong instantiation results:
Shape:  ()
NDim:  0


We've instantiated an array by calling the `np.array()` function on an iterable. This will create a one-dimensional numpy array, or _vector_. This is the first _important_ difference from MATLAB. In MATLAB, all arrays are by default n-dimensional. If you write:
```matlab
a = 1
a(:, :, :, :, :, 1) = 2
```
then you just received a 6-dimensional array. Numpy doesn't allow that. The dimensions of an array are generally the dimensions it had when it was created. You can always use `reshape`, like MATLAB, to define the exact number of dimensions you wish to have. You can also use `np.newaxis` to add an axis, as we'll see below. But these options still keep the number of elements in the array the same, they just re-order them.

This means that a 2x4 array can be reshaped into a 4x2 array, or a 2x2x2 one, but not 3x3x3.

The second difference is the idea of vectors.

In [4]:
print(f"The number of dimensions is {arr.ndim}.")
print(f"The shape is {arr.shape}.")

The number of dimensions is 1.
The shape is (3,).


As you see, `ndim` returns the number of dimensions. The `shape` is returned as a tuple, the length of which is the number of dimensions, while the values represent the number of elements in each dimensions. `shape` is very similar to the `size()` function of MATLAB.

Creating arrays with more than one dimension might look odd at a first glance:

In [5]:
arr_2d = np.array([[1, 2, 3], [4, 5, 6]])
arr_2d

array([[1, 2, 3],
       [4, 5, 6]])

In [6]:
print(f"The number of dimensions is {arr_2d.ndim}.")
print(f"The shape is {arr_2d.shape}.")
print(f"The len of the array is {len(arr_2d)} - which corresponds to the shape of the 0th dimension.")

The number of dimensions is 2.
The shape is (2, 3).
The len of the array is 2 - which corresponds to the shape of the 0th dimension.


As we see, each list was considered a single "row" (dimension number 0) in the final array, very much like MATLAB, although the syntax can be confusing at first. Here's another slightly confusing example:

In [7]:
c = np.array([[[1], [2]], [[3], [4]]])
c

array([[[1],
        [2]],

       [[3],
        [4]]])

In [8]:
c.shape

(2, 2, 1)

In [9]:
c2 = np.array([1, 2, 3, 4]).reshape((2, 2, 1))
c2.shape

(2, 2, 1)

### Getting help

Numpy has a ton of features, and to get around it we can use `lookfor` and the `?` sign, besides the official reference on the internet:

In [10]:
np.lookfor("Create array")

Search results for 'create array'
---------------------------------
numpy.array
    Create an array.
numpy.memmap
    Create a memory-map to an array stored in a *binary* file on disk.
numpy.diagflat
    Create a two-dimensional array with the flattened input as a diagonal.
numpy.fromiter
    Create a new 1-dimensional array from an iterable object.
numpy.partition
    Return a partitioned copy of an array.
numpy.ctypeslib.as_array
    Create a numpy array from a ctypes array or POINTER.
numpy.ma.diagflat
    Create a two-dimensional array with the flattened input as a diagonal.
numpy.ma.make_mask
    Create a boolean mask from an array.
numpy.ctypeslib.as_ctypes
    Create and return a ctypes object from a numpy array.  Actually
numpy.ma.mrecords.fromarrays
    Creates a mrecarray from a (flat) list of masked arrays.
numpy.ma.mvoid.__new__
    Create a new masked array from scratch.
numpy.lib.format.open_memmap
    Open a .npy file as a memory-mapped array.
numpy.ma.MaskedArray.__new_

In [71]:
np.con*?

### Creating arrays

In [12]:
np.arange(10)  # similar to MATLAB's 0:9 (and to Python's range())

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [13]:
np.arange(start=1, stop=8, step=3)  # MATLAB's 1:3:8

array([1, 4, 7])

In [14]:
np.zeros((2, 3))  # notice that the shape is a tuple

array([[0., 0., 0.],
       [0., 0., 0.]])

In [15]:
np.zeros_like(arr_2d)

array([[0, 0, 0],
       [0, 0, 0]])

In [16]:
np.ones((1, 4), dtype='int64')  # 2-d array with the dtype argument

array([[1, 1, 1, 1]])

In [17]:
np.full((4, 2), 7)

array([[7, 7],
       [7, 7],
       [7, 7],
       [7, 7]])

In [18]:
np.linspace(0, 10, 3)  # start, stop, number of points (endpoint as a keyword argument)

array([ 0.,  5., 10.])

In [19]:
np.linspace(0, 10, 3, endpoint=False, dtype=np.float32)  # float32 is "single" in MATLAB-speak. float64, which is the default, is "double"

array([0.       , 3.3333333, 6.6666665], dtype=float32)

In [20]:
np.eye(3, dtype='uint8')

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=uint8)

In [21]:
np.diag([19, 20])

array([[19,  0],
       [ 0, 20]])

In [22]:
# Some more interesting features
np.array([True, False, False])

array([ True, False, False])

In [23]:
np.array([1+2j]).dtype

dtype('complex128')

In [24]:
np.array(['a', 'bc', 'd', 'e'])  # <U2 are strings containing maximum 2 unicode letters
# No such thing as a cell array, arrays can contain strings.

array(['a', 'bc', 'd', 'e'], dtype='<U2')

In [25]:
# Arrays can be heterogeneous, although usually this is discouraged. 
# To make an hetetogeneous array, you have to specify its data type as "object", or "np.object":
np.array([1, 'b', True, {'b': 1}], dtype=object)

array([1, 'b', True, {'b': 1}], dtype=object)

The last few examples showed us that numpy arrays are a superset of MATLAB's matrices, as they also contain the cell array functionality within them.

When multiplying numpy arrays, multiplication is done cell-by-cell (elementwise), rather than by the rules of linear algebra. 

You can still multiply vectors and matrices using one of two options:
* The `@` operator (preferred): `arr1 @ arr2`
* `arr1.dot(arr2)`

Just remember that the default behavior is different than MATLAB's.

Also, numpy does contain a matrix-like array - `np.matrix('1 2; 3 4')` - which behaves like a linear algebra 2D matrix, but its use is discouraged.

### More on datatypes

While `numpy` doesn't require you to actively think of the data type of each of your arrays, it's a useful habit to have. The existing data types in `numpy` are:

In [26]:
np.sctypes

{'int': [numpy.int8, numpy.int16, numpy.int32, numpy.int64],
 'uint': [numpy.uint8, numpy.uint16, numpy.uint32, numpy.uint64],
 'float': [numpy.float16, numpy.float32, numpy.float64, numpy.float128],
 'complex': [numpy.complex64, numpy.complex128, numpy.complex256],
 'others': [bool, object, bytes, str, numpy.void]}

Many array creation and handling methods have a `dtype` keyword argument, that allow you to specify the required output's data type easily.

Python will issue RuntimeWarnings whenever an "unsafe" conversion of data type will occur, for example from `np.float64` to `np.float32`.

### Difference between numpy arrays and lists

You might have asked yourselves by this point what's the difference between lists and numpy arrays. In other words, if Python already has an array-like data structure, why does it need another one?

Lists are truly amazing, but their flexibility comes at a cost. You won't notice it when you're writing some short script, but when you start working with real datasets you notice that their performance is _lacking_.

Naive iteration over lists is good enough when the lists contain up to a few thousands of elements, but somewhere after this invisible threshold you will start to notice the runtimes of your app.

#### Why are lists slower?

Lists are slower due to their implementation. Because a list has to be heterogeneous, a list is actually an object containing references to other objects, which are the elements of the data contained in the list. This nesting can go on even deeper, since elements of lists can be other lists, or other complex data types.

Iterating over such a complicated objects contains a lot of "overhead", i.e. time the interpreter tries to figure out what is it actually facing - is this current value an integer? A float? A dictionary of tuples?

![Numpy arrays versus lists](extra_material/array_vs_list.png)

This is where numpy arrays come into the picture. They require the contained data to be homogeneous (disregarding the "object" datatype), leading to a simpler structure of the underlying implementation: A numpy array is an object with a pointer to a contiguous block of data in the memory.

Numpy forces the elements in its array to be homogeneous, by means of "upcasting":

In [27]:
arr = np.array([1, 2, 3.14, 4])
arr  # upcasted to float, not "object", since it's much slower

array([1.  , 2.  , 3.14, 4.  ])

This homogeneity and contiguity allows numpy to use "vectorized" functions, which are simply functions that can iterate in C code (and SIMD) over the array, as opposed to regular functions which have to use the Python iteration rules.

This means that while in essence the following two pieces of code are identical, the performance gain is due to the loop being done in C rather than in Python (this is the case in MATLAB as well):

In [28]:
%%timeit
python_array = list(range(1_000_000))
for item in python_array:
    item += 1

95 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%%timeit 
numpy_array = np.arange(1_000_000)
numpy_array += 1  # inside, a C loop is adding 1 to each item.

# Two orders of magnitude improvement for a pretty small (1M elements) array. 
# This is approximately the size of a 1024x1024 pixel image.

2.93 ms ± 644 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


If you recall the first class, you'll remember that we mentioned how Python code has to be transpiled into C code, which is then compiled to machine code. Numpy arrays take a "shortcut" here, are are quickly compiled to efficient C code without the Python overhead. When using vectorized numpy operations, the loop Python does in the backstage is very similar to a loop that a C programmer would have written by hand.

Another small but significant benefit of numpy arrays is smaller memory footprint:

In [30]:
from sys import getsizeof

python_array = list(range(1_000_000))
list_size = getsizeof(python_array) / 1e6  # in MB
print(f"Python list size (1M elements, MB): {list_size}")

numpy_array = np.arange(1_000_000, dtype=np.int32)
numpy_size = numpy_array.nbytes / 1e6
print(f"Numpy array size (1M elements, MB): {numpy_size}")

# Why a million elements take 4 MB? Each element has to weigh 4 bytes, or 32 bits. This means that the
# np.arange() function generates by default int32 values.

Python list size (1M elements, MB): 9.000112
Numpy array size (1M elements, MB): 4.0


Lower memory usage can help us pack larger array in the computer's working memory (RAM), and it will also improve the performance of the underlying code due to caching.

### Indexing and slicing

Indexing into `numpy` arrays is very much what you'd expect it to be.

In [31]:
a = np.arange(10)
print(f"a = {a}")
a[0], a[3], a[-2]  # Python indexing is always done with square brackets

a = [0 1 2 3 4 5 6 7 8 9]


(0, 3, 8)

In [32]:
# The beautiful reverse slicing is here as well
a[::-1]

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

In [33]:
arr_2d

array([[1, 2, 3],
       [4, 5, 6]])

In [34]:
arr_2d[0, 2]  # first row, third column

3

In [35]:
arr_2d[0, :]  # first row, all columns

array([1, 2, 3])

In [36]:
arr_2d[0]  # first item in the first dimension, and all of its corresponding elemets
# Similar to arr_2d[0, :]

array([1, 2, 3])

In [37]:
a[1::2]

array([1, 3, 5, 7, 9])

In [38]:
a[:2]  # last index isn't included

array([0, 1])

In [39]:
# In Python, slicing creates a view of an object, not a copy:
b = a[::2]
b

array([0, 2, 4, 6, 8])

In [40]:
b[0] = 100
b

array([100,   2,   4,   6,   8])

In [41]:
a  # a is also changed!

array([100,   1,   2,   3,   4,   5,   6,   7,   8,   9])

In [42]:
# Copying is done with .copy()
a_copy = a[::-1].copy()
a_copy

array([  9,   8,   7,   6,   5,   4,   3,   2,   1, 100])

In [43]:
a_copy[1] = 200
a_copy

array([  9, 200,   7,   6,   5,   4,   3,   2,   1, 100])

In [44]:
a  # unchanged

array([100,   1,   2,   3,   4,   5,   6,   7,   8,   9])

This behavior allows Numpy to save time and memory. It will usually go unnoticed during day-to-day use. Occasionally, however, it can result in ugly bugs which are hard to locate.

Let's look at a summary of 2D indexing:

![Numpy indexing](extra_material/numpy_indexing.png)

### Aggregation

In [45]:
arr = np.random.random(1_000)
sum(arr)  # The native Python sum works on numpy arrays, but its use is discouraged in this context

492.6043817490305

`sum()` is a built in Python function, capable of working on lists and other native Python data structures. Numpy has its own sum functions: You can either write `np.sum(arr)`, or use `arr.sum()`:

In [46]:
%timeit sum(arr)
%timeit np.sum(arr)
%timeit arr.sum()

300 µs ± 31.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
6.45 µs ± 745 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
4.9 µs ± 568 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Keep in mind that the built-in `sum()` and numpy's `np.sum()` aren't identical. `np.sum()` by default calculates the sum of _all axes_, returning a single number for multi-dimensional arrays.

In MATLAB, this behavior is replicated with `sum(arr(:))`.

Likewise, `min()` and `max()` also have two "competing" versions:

In [47]:
%timeit min(arr)  # Native Python
%timeit np.min(arr)
%timeit arr.min()

161 µs ± 9.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
5.73 µs ± 451 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.63 µs ± 299 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Calculating the `min` of an array will, again, result in a single number:

In [48]:
arr2 = np.random.random((100, 100, 100))
arr2.min()

5.823232201995765e-07

If you wish to calculate it across an axis, use the `axis` keyword:

In [49]:
# 2D output - axis number 0 was "dropped"
print(arr2.min(axis=0))

print('------')
# 1D output - the two first axes were summed over
print(arr2.max(axis=(0, 1)))

[[1.30105370e-04 1.85247986e-02 5.76981265e-04 ... 2.17054628e-03
  1.74017353e-03 1.71648417e-02]
 [9.74708434e-04 1.42110375e-02 2.27249914e-04 ... 1.84564336e-04
  9.91460849e-03 5.03250994e-04]
 [2.10501609e-03 3.11640007e-03 9.31462323e-03 ... 4.80578692e-03
  2.53371128e-03 1.15754554e-02]
 ...
 [2.64927481e-03 2.05463086e-03 2.03506809e-02 ... 1.71246031e-03
  1.22983732e-02 1.23869152e-02]
 [6.90173657e-04 3.43165101e-03 8.55869758e-03 ... 7.83710115e-04
  1.33042948e-02 7.60761611e-03]
 [3.24140890e-03 7.34097009e-03 9.79868560e-03 ... 3.08316230e-02
  6.87046457e-03 2.45869442e-06]]
------
[0.99996561 0.99998143 0.99979485 0.99995817 0.99992053 0.99996168
 0.9997315  0.99988793 0.99991571 0.9999547  0.99999641 0.99999241
 0.99995579 0.99997798 0.99991344 0.99989906 0.9997946  0.9999751
 0.99997769 0.99986947 0.99986118 0.99993987 0.99994818 0.99999912
 0.99978118 0.999931   0.99992138 0.9997643  0.99999697 0.99999074
 0.99997707 0.9998866  0.99990193 0.99990738 0.99998946 0.9

Many other aggregation functions exist, including:
```
- np.var
- np.std
- np.argmin\argmax
- np.median
- ...
```
Most of them have an object-oriented version, i.e. `arr.var()`, and a procedural version, i.e. `np.var(arr)`.

### Fancy indexing

In Numpy, indexing an array with a different array is called "fancy" indexing. Perhaps confusingly, it creates a copy, not a view.

In [50]:
basic_array = np.arange(10)
basic_array

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [51]:
mask = (basic_array % 2 == 0)
mask

array([ True, False,  True, False,  True, False,  True, False,  True,
       False])

In [52]:
b = basic_array[mask]  # also basic_array[basic_array % 2 == 0]
b

array([0, 2, 4, 6, 8])

MATLAB veterans shouldn't be surprised from this feature, but if you haven't seen it before make sure to have a firm grasp on it, as it's a very powerful technique.

In [53]:
b[:] = 999
print(f"basic array: {basic_array}")
print(f"b: {b}")

basic array: [0 1 2 3 4 5 6 7 8 9]
b: [999 999 999 999 999]


In [54]:
float_array = np.arange(start=0, stop=20, step=1.5)
float_array

array([ 0. ,  1.5,  3. ,  4.5,  6. ,  7.5,  9. , 10.5, 12. , 13.5, 15. ,
       16.5, 18. , 19.5])

In [55]:
float_array[[1, 2, 5, 5, 10]]  # copy, not a view. Meaning that the resulting array is a new
# instance of the original array, independent of it, in a different location in memory.

array([ 1.5,  3. ,  7.5,  7.5, 15. ])

Counting the amount of values that fit our condition can be done as follows:

In [56]:
basic_array = np.arange(10)
count_nonzero_result = np.count_nonzero(basic_array % 2 == 0)
sum_result = np.sum(basic_array % 2 == 0)

print(f"np.count_nonzero result: {count_nonzero_result}")
print(f"np.sum result: {sum_result}")
# In the latter case, True is 1 and False is 0

np.count_nonzero result: 5
np.sum result: 5


If we wish to add more conditions in to the mix, we can use the `&, |, ~, ^` operators (and, or, not, xor):

In [57]:
((basic_array % 2 == 1) & (basic_array != 3)).sum()  # uneven values that are different from 3

4

Note that numpy uses the `&, |, ~, ^` operators for element-by-element comparison, while the reserved `and` and `or` keywords evaluate the entire object:

In [58]:
np.sum((basic_array % 2 == 1) and (basic_array != 3))   # doesn't work, "and" isn't used this way here

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

### Sorting

Again we face a condition in which Python has its own `sort` functions, but numpy's `np.sort()` are much better suited for arrays. We can either sort in-place, or have a new object back:

In [59]:
# Return a new object:
arr = np.array([ 3, 4, 1, 8, 10, 2, 4])
print(np.sort(arr))

# Sort in-place:
arr.sort()
print(arr)

[ 1  2  3  4  4  8 10]
[ 1  2  3  4  4  8 10]


The default implementation is a _quicksort_ but other sorting algorithms can be found as well.

In [60]:
# np.argsort will return the indices of the sorted array:
arr = np.array([ 3, 4, 1, 8, 10, 2, 4])
print("Sorted indices: {}".format(np.argsort(arr)))

# Usage is as follows:
arr[np.argsort(arr)]

Sorted indices: [2 5 0 1 6 3 4]


array([ 1,  2,  3,  4,  4,  8, 10])

### Concatenation

Concatenation (and splitting) in numpy works as follows:

In [61]:
x = np.array([1, 2, 3])
y = np.array([3, 2, 1])
np.concatenate([x, y])  # concatenates along the first axis

array([1, 2, 3, 3, 2, 1])

In [62]:
# Concatenate more than two arrays
z = [99, 99, 99]
np.concatenate([x, y, z])  # notice how the function argument is an iterable!

array([ 1,  2,  3,  3,  2,  1, 99, 99, 99])

In [63]:
# 2D - along the last axis
twod_1 = np.array([[0, 1, 2],
                   [3, 4, 5]])
twod_2 = np.array([[6, 7, 8],
                   [9, 10, 11]])
np.concatenate((twod_1, twod_2), axis=-1)

array([[ 0,  1,  2,  6,  7,  8],
       [ 3,  4,  5,  9, 10, 11]])

In [64]:
# np.vstack and np.hstack are also an option
sim1 = np.array([0, 1, 2])
sim2 = np.array([[30, 40, 50],
                 [60, 70, 80]])
np.vstack((sim1, sim2))

array([[ 0,  1,  2],
       [30, 40, 50],
       [60, 70, 80]])

Splitting up arrays is also quite easy:

In [65]:
arr = np.arange(16).reshape((4, 4))  # see the tuple? Shapes of arrays always come in tuples
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [66]:
upper, lower = np.split(arr, [3])  # splits at the fourth line (of the first axis, by default)
print(upper)
print('---')
print(lower)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
---
[[12 13 14 15]]


### Exercise

#### a. Familiarization:

Create two random 3D arrays, at least 3x3x3 in size. The first should have random integers between 0 and 100, while the second should contain floating points numbers drawn from the normal distribution, with mean -1 and standard deviation of 2 

1. What are the default datatypes of integer and floating-point arrays in numpy?

2. Center the distribution of the first integer array around 0, with values between -1 and 1.

3. Caclulate their mean, standard deviation along the last axis, and sum along the second axis. Remember that arrays are, like everything in Python, objects.


#### b. _Iteration over an array:_ 

Find and return the indices of the two random arrays only where _both elements_ comply with -0.5 <= X <= 0.5. Do so in at least two distinct ways. These can include elementwise iteration, masked-arrays, and more.

Time the execution of the script using the IPython `%%timeit` magic if you have this tool near at hand.

#### c. _Broadcasting:_
1. Create a vector with length 100, filled with ones, and a 2D zero-filled array with its last dimension with size of 100. 
2. Using `np.tile`, add the vector and the array.
3. Did you have to use `np.tile` in this case? What feature of numpy allows this?
4. What happens to the non-tiled addition when the first dimension of the matrix is 100? Why?
5. Bonus: How can one add the matrix (in its new shape) and vector without np.tile()?

### Exercise solutions below...

In [67]:
# a
# 1
low = 0
high = 100
arr1 = np.random.randint(low=low, high=high, size=(10, 10, 10))  # dtype is np.int32
arr2 = np.random.randn(10, 10, 10) * 2 - 1  # dtype is np.float64

# 2
middle = (high - low) / 2
arr1 = (arr1 - middle) / middle

# 3
print("arr1 (integers) stats:")
print(f"Mean: {arr1.mean()},\nSTD: {arr1.std(axis=-1)},\nSum: {arr1.sum(axis=1)}")
print("arr2 (normal distribution) stats:")
print(f"Mean: {arr2.mean()},\nSTD: {arr2.std(axis=-1)},\nSum: {arr2.sum(axis=1)}")

arr1 (integers) stats:
Mean: -0.022420000000000002,
STD: [[0.46527841 0.58366086 0.4970674  0.52520472 0.60564346 0.63283805
  0.46085139 0.4209513  0.4701872  0.38631076]
 [0.43277708 0.36011665 0.54919578 0.53052427 0.56936807 0.59547964
  0.67530438 0.49422262 0.59702261 0.50169712]
 [0.61430937 0.63617608 0.50404762 0.29719354 0.52853004 0.59573148
  0.35097578 0.55072679 0.4445942  0.70312446]
 [0.50732238 0.60033657 0.59707621 0.61442656 0.5200923  0.54035544
  0.6159513  0.34816663 0.44210406 0.44297178]
 [0.6061155  0.57822141 0.44680645 0.45090576 0.41950447 0.66157388
  0.56838367 0.47866899 0.43709953 0.30857738]
 [0.46726866 0.58472558 0.5181853  0.65363292 0.62387178 0.45847137
  0.62560371 0.5420738  0.63403785 0.4835866 ]
 [0.63245869 0.63586477 0.44535379 0.4886553  0.67346566 0.59189188
  0.65740399 0.67680426 0.63917447 0.49990399]
 [0.62806369 0.47633602 0.54065146 0.47453556 0.56703086 0.64226163
  0.37139736 0.57250677 0.46189176 0.5343744 ]
 [0.51646878 0.36351616

In [68]:
%%timeit
# b
# 1 - elementwise
result = []
for idx, (item1, item2) in enumerate(zip(arr1.flat, arr2.flat)):
    if (-0.5 <= item1 <= 0.5) and (-0.5 <= item2 <= 0.5):
        result.append(idx)
result

670 µs ± 92.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [69]:
%%timeit
# b
# 2 - masked arrays createed by np.ravel(), np.where()
mask1 = ((-0.5 <= arr1.ravel()) & (arr1.ravel() <= 0.5))
mask2 = ((-0.5 <= arr2.ravel()) & (arr2.ravel() <= 0.5))
both_positive = np.logical_and(mask1, mask2)
result = np.where(both_positive)
result

17.8 µs ± 3.26 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


As you can tell, the second solution is far more efficient than the first, and after a few read code blocks, the syntax presented there can be at least as clear as the one in the first solution.

In [70]:
# c
# 1
vec = np.ones(100)
mat = np.zeros((10, 100))

# 2
tiled_vec = np.tile(vec, (10, 1))  # transpose is necessary
ans_tiled = mat + tiled_vec

# 3
ans_simple = mat + vec  # It's called array broadcasting

# 4
mat_transposed = mat.T
print(mat_transposed.shape)
# ans_trans = mat_transposed + vec  # fails because broadcasting is done over the last dimension

# 5 - np.newaxis is the answer - very perfomant
ans_newaxis = mat_transposed + vec[:, np.newaxis]

(100, 10)
